In [7]:
import utils as u
import torch

train, test = u.get_data()
train = u.preprocess_data(train, True)
test = u.preprocess_data(test, False)

X_train, X_valid, y_train, y_valid = u.train_validation_split(train, 0.33)
x_train, y_train, x_valid, y_valid = map(torch.tensor, (X_train, y_train, X_valid, y_valid))
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)
print(test.shape)

torch.Size([30269, 40])
torch.Size([14909, 40])
torch.Size([30269])
torch.Size([14909])
(6931, 40)


In [8]:
from torch.utils.data import TensorDataset, DataLoader
from ANN import ANN

bs = 400
lr = 0.5

x_train = x_train.float()
y_train = y_train.long()
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

x_valid = x_valid.float()
y_valid = y_valid.long()

def get_model():
    model = ANN()
    return model, torch.optim.SGD(model.parameters(), lr=lr)

def accuracy(out, y):
    out = torch.argmax(out, dim=1)
    return (out == y).float().mean()

In [19]:
import torch.nn.functional as F

epochs = 5
model, opt = get_model()
loss_func = F.cross_entropy

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    model.eval()
    with torch.no_grad():
        valid_loss = loss_func(model(x_valid), y_valid)
    
    print(epoch, valid_loss, accuracy(model(x_valid), y_valid))
    

0 tensor(0.4635) tensor(0.7756)
1 tensor(0.4359) tensor(0.7952)
2 tensor(0.5216) tensor(0.7535)
3 tensor(0.4455) tensor(0.7882)
4 tensor(0.4198) tensor(0.8051)
